In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/gamestop-historical-stock-prices/GME_stock.csv', index_col='date')
df = df.sort_values('date', ascending=True)
print(df)

In [ ]:
import plotly.graph_objs as go
#create the figure 

fig = go.Figure()

fig.add_trace(
    go.Scatter(x= df.index, y= df.close_price ))

#set title
fig.update_layout(
    title_text = 'Time series with range slider and selectors'
        )

#Add range slider 
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                     label='YTD',
                     step='year',
                     stepmode='todate'),
                dict(count=1,
                     label='1y',
                     step='year',
                     stepmode='backward'),
                dict(count=5,
                     label='5y',
                     step='year',
                     stepmode='backward'),                
                dict(step='all')                             
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type='date'
    )

)

fig.show()

In [ ]:
import math
#Creat a new dataframe only close price 
close_price= df.filter(['close_price'])
#Covert the dataframe to numpy array
data_set= close_price.values
data_set

In [ ]:
#Get the 80% close price value to train the model.
training_data= math.ceil(len(data_set) * 0.8)
training_data

In [ ]:
data_set.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_set = data_set.reshape(-1, 1)
scaled_data = scaler.fit_transform(data_set)
scaled_data

In [ ]:
#Create the training data set
#Create the scaled training data set
train_data = scaled_data[0:training_data, :]
#Split the data into x_train and y_train data sets
x_train=[]
y_train=[]

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i <=61:
        print(x_train)
        print(y_train)
        print()
        

In [ ]:
#Convert the x_train and y_train to numpy arrays
x_train, y_train= np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

#Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compile the model 
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Train the model 
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
#Create the testing data set
#Create a new array containing scaled values

test_data = scaled_data[training_data -60: , :]
#Create the data sets x_test and y_test
x_test = []
y_test = data_set[training_data: , :]

for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    

In [ ]:
#Convert the data to a numpy array
x_test = np.array(x_test)
x_test

In [ ]:
#Reshape the data 
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get the models predicted price values
predictions = model.predict(x_test)
predicitons = scaler.inverse_transform(predictions)


In [ ]:
#Get the root mean squared error(RMSE)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
import matplotlib.pyplot as plt 
#Plot the data
train = close_price[:training_data]
valid = close_price[training_data:]
valid['Predictions'] = predicitons
plt.figure(figsize=(16, 8))
plt.title('Model')
plt.xlabel('Data', fontsize=18)
plt.ylabel('Close Price', fontsize=18)
plt.plot(train['close_price'])
plt.plot(valid[['close_price', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
#Show the valid and prediction prices
valid

In [ ]:
#Get the quote
new_df = pd.read_csv('/kaggle/input/gamestop-historical-stock-prices/GME_stock.csv', index_col='date')
new_df = df.sort_values('date', ascending=True)
new_df= df.filter(['close_price'])
#Get the last 60 day closing prices values and convert the dataframe to an array
last_60_days = new_df[-60:].values
#Scale the data to be value between 0 and 1
last_60_days_scaled= scaler.transform(last_60_days)
#Creat an empty list
X_test=[]
#Append the past 60 days
X_test.append(last_60_days_scaled)
#Covert the X_test set to a numpy array
X_test= np.array(X_test)
#Reshape the data
X_test= np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
print('2021-01-28 close price:',new_df.tail(1).values)